# Getting started with Spark: demo with Spark SQL in Python


This demo is based on [Spark SQL Guide - Getting started](https://spark.apache.org/docs/latest/sql-getting-started.html).

For this demo we used the city of Vienna trees dataset ("Baumkataster") made available by [Open Data Österreich](https://www.data.gv.at) and downloadable from [here](https://www.data.gv.at/katalog/dataset/c91a4635-8b7d-43fe-9b27-d95dec8392a7) .

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

Read the file into a Spark [_dataframe_](https://spark.apache.org/docs/latest/sql-programming-guide.html#datasets-and-dataframes).

In [3]:
df = spark.read \
          .load("/data/FME_BaumdatenBearbeitet_OGD_20190205.csv",
           format="csv", sep=";", header="true", encoding="iso-8859-1")

**Note:** The directory `/data` resides on the Hadoop distributed filesystem. If you want to upload your own CSV file `myFile.csv` you need to first upload it to HDFS like this:
<code>
    hdfs dfs -put myFile.csv myFile.csv
</code>
and then you can just use `.load("myFile.csv", ...` in the Spark load command.

Show first three lines of Spark dataframe

In [4]:
df.show(3)

+-------+------+------------+----------+-----+-----------+-----+-----------------+-----------+---+-----+------+----------------+----------------+
|Flaeche|BaumNr|     Gattung|       Art|Sorte|NameDeutsch|Hoehe|Schirmdurchmesser|Stammumfang|Typ| XPos|  YPos|             lon|             lat|
+-------+------+------------+----------+-----+-----------+-----+-----------------+-----------+---+-----+------+----------------+----------------+
|      0|     0|   ZumTesten|         0|    0|   20190205|    0|                0|          0|  0|70000|350000|14,2757549011314|48,2844031941042|
|    870|  021a|     Quercus|       sp.|    -|      Eiche|    6|                3|         31|  L|72431|354949|14,3093549528352|48,3286271802142|
|    572|   127|Liriodendron|tulipifera|    -| Tulpenbaum|    5|                2|         21|  L|71171|353742|14,2921648325343|48,3179178510249|
+-------+------+------------+----------+-----+-----------+-----+-----------------+-----------+---+-----+------+-------------

For pretty-printing you can use `toPandas()`

In [5]:
df.toPandas().head(3)

,Flaeche,BaumNr,Gattung,Art,Sorte,NameDeutsch,Hoehe,Schirmdurchmesser,Stammumfang,Typ,XPos,YPos,lon,lat
0,0,0,ZumTesten,0,0,20190205,0,0,0,0,70000,350000,"14,2757549011314","48,2844031941042"
1,870,021a,Quercus,sp.,-,Eiche,6,3,31,L,72431,354949,"14,3093549528352","48,3286271802142"
2,572,127,Liriodendron,tulipifera,-,Tulpenbaum,5,2,21,L,71171,353742,"14,2921648325343","48,3179178510249"


Show number of different trees (count german names in `df` and sort by cont)

In [6]:
df.groupBy("NameDeutsch").count().orderBy('count', ascending=False).show()

+--------------------+-----+
|         NameDeutsch|count|
+--------------------+-----+
|        Winter-Linde| 1583|
|          Weiß-Birke| 1442|
|         Spitz-Ahorn| 1273|
|         Stiel-Eiche| 1228|
|               Ahorn| 1079|
|           Hainbuche| 1036|
|       Gemeine Esche|  987|
|Ahornblättrige-Pl...|  961|
|            Rotbuche|  747|
|               Linde|  688|
|          Feld-Ahorn|  637|
|          Berg-Ahorn|  566|
|    Säulen-Hainbuche|  507|
|Gemeine Rosskastanie|  461|
|          Tulpenbaum|  436|
|             Robinie|  358|
|        Silber-Weide|  353|
|      Schwarz-Kiefer|  321|
|           Baumhasel|  258|
|    Serbische Fichte|  257|
+--------------------+-----+
only showing top 20 rows



An example of SQL query (see [Running SQL Queries Programmatically](https://spark.apache.org/docs/latest/sql-getting-started.html#running-sql-queries-programmatically)): let's sort trees by height ("Hoehe").

In [7]:
df.createOrReplaceTempView("baeume")

In [8]:
spark.sql("SELECT BaumNr, NameDeutsch, Hoehe, lat, lon FROM baeume order  by Hoehe desc").show()

+------+--------------------+-----+----------------+----------------+
|BaumNr|         NameDeutsch|Hoehe|             lat|             lon|
+------+--------------------+-----+----------------+----------------+
|   844|            Weißdorn|   99|48,3198141692826|14,3032456456049|
|   005|           Hainbuche|   99|48,3107341721266|14,2818381176027|
|   129|          Weiß-Birke|   99|48,2764221420506|14,3029622419918|
|   037|Gemeine Kiefer / ...|   95|48,3021818905511|14,2769106844535|
|   007|          Tulpenbaum|   90|48,2765758955147|14,3027882173276|
|   051|          Weiß-Birke|    9|48,2671468323820|14,2832242240464|
|   001|    Säulen-Hainbuche|    9|48,2973425597313|14,3138560746343|
|   009|        Sommer-Linde|    9|48,2383321988402|14,3668869798697|
|   025|       Schwarz-Birke|    9|48,2749605071833|14,2781176169409|
|   004|            Rotbuche|    9|48,2768058741599|14,2821412254345|
|   011|             Robinie|    9|48,2382302739335|14,3347263036350|
|   009|          Gö

The height data doesn't seem to be up-to-date.

**Note:** After you're done, it's important to close the Spark session in order to release cluster resources.

In [9]:
spark.stop()